In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
# 来源：中科院硕士论文-闻博
# 在自己的数据集上复现其模型

# 0. TPU准备

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

# 1. 载入数据

In [ ]:
train_data = pd.read_csv(r'../input/blood-pressure-datasets/Train_Merge_Data.csv')
validation_data = pd.read_csv(r'../input/blood-pressure-datasets/Validation_Merge_Data.csv')

In [ ]:
train_data = shuffle(train_data)
validation_data = shuffle(validation_data)

In [ ]:
train_label_Sbp = train_data.iloc[:,610]
train_label_Sbp = train_label_Sbp.values

train_label_Dbp = train_data.iloc[:,611]
train_label_Dbp = train_label_Dbp.values

train_data = train_data.iloc[:,:600]
train_data = train_data.values

In [ ]:
validation_label_Sbp = validation_data.iloc[:,610]
validation_label_Sbp = validation_label_Sbp.values

validation_label_Dbp = validation_data.iloc[:,611]
validation_label_Dbp = validation_label_Dbp.values

validation_data = validation_data.iloc[:,:600]
validation_data = validation_data.values

In [ ]:
print("train_data_information:")
print(train_data.shape)
print(train_label_Sbp.shape)
print(train_label_Dbp.shape)
print("validation_data_information:")
print(validation_data.shape)
print(validation_label_Sbp.shape)
print(validation_label_Dbp.shape)

# 2. 自定义模型模块

In [ ]:
def Conv1d_ResNet_Block(input_layer,filters=64,kernel_size=3,strides=1,connect_strides=2,padding='same',start=False):
    
    if start!=False:
        #1
        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(input_layer)
        layer = layers.BatchNormalization()(layer)
        layer = layers.Activation(tf.nn.relu)(layer)

        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(layer)
        layer = layers.BatchNormalization()(layer)

        resnet_layer = layers.BatchNormalization()(input_layer)
        layer = layers.add([layer,resnet_layer])

        layer = layers.Activation(tf.nn.relu)(layer)
        #2
        input_layer = layer
        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(input_layer)
        layer = layers.BatchNormalization()(layer)
        layer = layers.Activation(tf.nn.relu)(layer)

        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(layer)
        layer = layers.BatchNormalization()(layer)

        resnet_layer = layers.BatchNormalization()(input_layer)
        layer = layers.add([layer,resnet_layer])

        layer = layers.Activation(tf.nn.relu)(layer)
    else:
        #1
        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=connect_strides,padding=padding)(input_layer)
        layer = layers.BatchNormalization()(layer)
        layer = layers.Activation(tf.nn.relu)(layer)

        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(layer)
        layer = layers.BatchNormalization()(layer)
        
        resnet_layer = layers.Conv1D(filters=filters,kernel_size=1,strides=connect_strides,padding=padding)(input_layer)
        resnet_layer = layers.BatchNormalization()(resnet_layer)
        layer = layers.add([layer,resnet_layer])

        layer = layers.Activation(tf.nn.relu)(layer)
        #2
        input_layer = layer
        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(input_layer)
        layer = layers.BatchNormalization()(layer)
        layer = layers.Activation(tf.nn.relu)(layer)

        layer = layers.Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding)(layer)
        layer = layers.BatchNormalization()(layer)

        
        resnet_layer = layers.BatchNormalization()(input_layer)
        layer = layers.add([layer,resnet_layer])

        layer = layers.Activation(tf.nn.relu)(layer)
    return layer


# 3. 模型框架

In [ ]:
def create_model():
    inputs = keras.Input(shape=(600,1))

    layer = layers.Conv1D(filters=64,kernel_size=7,strides=1,padding='same')(inputs)

    layer = layers.MaxPooling1D(pool_size=3,strides=2)(layer)

    layer = Conv1d_ResNet_Block(layer,filters=64,kernel_size=3,strides=1,connect_strides=2,padding='same',start=True)

    layer = Conv1d_ResNet_Block(layer,filters=128,kernel_size=3,strides=1,connect_strides=2,padding='same',start=False)

    layer = Conv1d_ResNet_Block(layer,filters=256,kernel_size=3,strides=1,connect_strides=2,padding='same',start=False)

    layer = Conv1d_ResNet_Block(layer,filters=512,kernel_size=3,strides=1,connect_strides=2,padding='same',start=False)

    layer = layers.GlobalAveragePooling1D()(layer)
    
    layer = layers.Dense(512,activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.4))(layer)
    layer = layers.Dropout(0.5)(layer)
    outputs_sbp = layers.Dense(1,name='Sbp')(layer)
    outputs_dbp = layers.Dense(1,name='Dbp')(layer)

    model = keras.Model(inputs=inputs,outputs=[outputs_sbp,outputs_dbp])
    
    return model

# 4. 评价指标

In [ ]:
"""自定义评价指标模块"""
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(y_pred-y_true)
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_pred-y_true) - u)))

# 5. 回调函数

In [ ]:
"""回调函数"""
#保存迭代周期内最好的模型
checkpoint_filepath = r'./model_struction.h5'
Save_epochs = 10 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_mean_absolute_error',
    mode='min',
    save_best_only=True)

# 6. 模型编译

In [ ]:
with strategy.scope():
    model = create_model()
    
    model.compile(loss={'Sbp':"mse",'Dbp':"mse"}, optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])

# 7. 模型图

In [ ]:
model.summary()

In [ ]:
"""保存模型结构图片"""
tf.keras.utils.plot_model(model, to_file=r'./model_graph.png', show_shapes=True, show_layer_names=False)

# 8. 训练模型

In [ ]:
history = model.fit(train_data,{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=(validation_data,{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )

In [ ]:
with open(r'./last_model.h5.pickle', 'wb') as file_pi:
 	pickle.dump(history.history, file_pi)